In [14]:
import logging
from typing import Any, Dict, List, Optional

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.aio import SearchClient
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
    VectorizedQuery,
)

class AzureAISearch:
    """
    Búsqueda híbrida (texto + vector). Si proporcionas semantic_config_name,
    usa SEMANTIC; de lo contrario, usa el query simple por defecto.
    """

    def __init__(
        self,
        endpoint: str,
        api_key: str,
        *,
        embedding_function: Optional[Any] = None,
        vector_field: str = "content_vector",
        semantic_config_name: Optional[str] = None,
    ):
        self.search_endpoint = endpoint
        self.search_credential = AzureKeyCredential(api_key)
        self.embedding_function = embedding_function
        self.vector_field = vector_field
        self.semantic_config_name = semantic_config_name

    async def hybrid_search(
        self,
        query: str,
        index_name: str,
        *,
        filter_str: Optional[str] = None,
        k: int = 5,
        select: Optional[List[str]] = None,
    ) -> List[Dict[str, Any]]:
        try:
            if not query:
                logging.error("La consulta no puede estar vacía")
                return []

            vector_query = None
            if self.embedding_function:
                embedding = self.embedding_function.embed_query(query)
                vector_query = VectorizedQuery(
                    vector=embedding,
                    k_nearest_neighbors=k,
                    fields=self.vector_field,
                )

            async with SearchClient(
                endpoint=self.search_endpoint,
                index_name=index_name,
                credential=self.search_credential,
            ) as search_client:
                kwargs = {
                    "search_text": query,
                    "vector_queries": [vector_query] if vector_query else None,
                    "filter": filter_str,
                    "top": k,
                }
                if select:
                    kwargs["select"] = select

                # Solo activar SEMANTIC si realmente existe el nombre
                if self.semantic_config_name:
                    kwargs.update(
                        dict(
                            query_type=QueryType.SEMANTIC,
                            semantic_configuration_name=self.semantic_config_name,
                            query_caption=QueryCaptionType.EXTRACTIVE,
                            query_answer=QueryAnswerType.EXTRACTIVE,
                        )
                    )

                results = await search_client.search(**kwargs)

                hits: List[Dict[str, Any]] = []
                async for r in results:
                    hits.append(dict(r))
                return hits

        except Exception as e:
            logging.exception(f"Error en búsqueda híbrida: {str(e)}")
            return []

def shape(doc, fields=None):
    out = {
        "id": doc.get("id"),
        "score": doc.get("@search.score"),
        "rerankerScore": doc.get("@search.rerankerScore"),
    }
    if doc.get("@search.captions"):
        out["caption"] = doc["@search.captions"][0].get("text")
    if doc.get("@search.answers"):
        out["answer"] = doc["@search.answers"][0].get("text")
    if fields:
        out["fields"] = {f: doc.get(f) for f in fields}
    return out


In [5]:
from openai import AzureOpenAI

class AzureOpenAIEmbeddings:
    def __init__(self, endpoint, api_key, deployment_name):
        self.client = AzureOpenAI(
            api_key=api_key,
            azure_endpoint=endpoint,
            api_version="2023-05-15"
        )
        self.deployment = deployment_name

    def embed_query(self, text: str):
        resp = self.client.embeddings.create(
            model=self.deployment,
            input=text
        )
        return resp.data[0].embedding  # list[float] de 1536 elementos


In [13]:
embedder = AzureOpenAIEmbeddings(
    endpoint="https://oai-general.openai.azure.com/",
    api_key="F95F4KJut2ku0kYGn54ZgTeH5nBaJWo7quHnD8748VT8DLx0xHHEJQQJ99BGACHYHv6XJ3w3AAABACOGfNvD",
    deployment_name="text-embedding-ada-002"  # nombre de tu deployment
)

search = AzureAISearch(
    endpoint="https://service-se.search.windows.net",
    api_key="IZFL3I5MIPtajkRY5WyFX8ELQkrcbtwq8rwaXHOUeRAzSeCVLLbG",
    embedding_function=embedder,
    semantic_config_name="azureml-default",
    vector_field="contentVector"
)

hits = await search.hybrid_search(
    query="Tramo ballena la mami",
    index_name="magenta-sand-46qlhmn39w",
    k=3,
    select=["id","content","title","url","filepath","meta_json_string"]  
)

hits


[{'content': "Title: CONTRATO  P-GC-CI-2024-003.pdf \n \n \n \nPágina  2 de 5 \n  \n                  \nGerencia Comercial de Transporte               _________                            \n  \n  Nombre del Punto \nde Salida  Ubicación  \n1. Ubicados en los \ntramos        \n \n4. Remitente Primario:  es Gases del Caribe S.A.E.S.P. . \n \n5. Actualización de Tarifa: La tarifa se encuentra expresada en pesos colombianos \nde 2021. \n \n6. Día Hábil:  Todos los Días de la semana, exceptuando domingos y festivos no \nlaborables, de acuerdo con la ley colombiana, y, exceptuando también, aquellos \nDías que son festivos en la ciudad de Barranquilla  , determinados \nlocalmente por eventos como ferias, carnavales, entre otros. \n \n7. Tramos:  El(Los) tramo(s) aplicable(s) al Servicio es(son):     Ballena - La Mami; \nLa Mami - Barranquilla, Barranquilla-Cartagena.   . \n \nDuración del Servicio: La duración del Servicio será, desde el 01 de junio  de  \n2024  hasta el 31 de agosto  de 2024 